# Imports

In [3]:
import boto3
import json
import numpy as np
from keras.models import load_model

# Utility Classes

In [4]:
class AWSUtilsLib():
    
    def __init__(self, aws_access_key, aws_secret_key, region_id = ''):
        self.sqs                    = None
        self.queue                  = None
        self.acess_key              = aws_access_key
        self.secret_key             = aws_secret_key
        self.region                 = region_id
        
    #######################
    #### SQS FUNCTIONS ####
    #######################
    def create_sqs_client(self, region_id = ''):
        
        if region_id:
            self.region = region_id
            
        self.sqs = boto3.resource('sqs',
             aws_access_key_id     = self.acess_key,
             aws_secret_access_key = self.secret_key,
             region_name           = self.region)

    def create_queue_client(self, queuename):

        # Get Queue URL
        queue_url = self.sqs.get_queue_by_name(QueueName=queuename)

        # Create Queue Resource
        self.queue = sqs.Queue(queue_url.url)
        

        
        


# Code

In [21]:
# Get sqs and queue client
aws = AWSUtilsLib('access','secret', 'us-east-1')
aws.create_sqs_client()
aws.create_queue_client('LUNAR_Processed')

# Load machine learning model
model = load_model('LUNAR_MachineLearningModel.h5')


datas   = []
samples = []
counter = 0

while True:
    
    # Get messages
    messages = queue.receive_messages(MaxNumberOfMessages=10, WaitTimeSeconds=5, AttributeNames=['All'])
    
    # Iterate over messages
    for message in messages:

        # Trace msg
        if counter % 60 == 0:
            print ('Already processed {} messages'.format(counter))
        
        # Deserialize object
        data = json.load(message.body)
        
        samples.append([data['Accelerometer_X'],
                        data['Accelerometer_Y'],
                        data['Accelerometer_Z']])
        
        if (len(samples) == 60):
            
            # Restructure the object to be readable by the saved model
            restructured_obj = [samples]
            
            # Applies the machine learning model
            y_pred = model.predict(restructured_obj)
            output = np.argmax(y_pred, axis=1)[0]
            
            # Save the predicted output on record
            data['Output'] = str(output)
        
            # Send to next queue (LUNAR_Processed)
        
        
            # Delete messages
            message.delete()
            
            samples = []
        
        counter += 1

SyntaxError: 'return' outside function (<ipython-input-21-abb634ce1f71>, line 2)